In [ ]:
# !pip install --upgrade pip

In [ ]:
# !python --version  
# Python 3.10.12

# Importing Libraries

In [ ]:
import os
import gc

import torch

import numpy as np
import pandas as pd

# !pip install sentencepiece

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

!pip install datasets

import datasets
from datasets import Dataset

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Globals

In [ ]:
SEED = 1337 # 45090448
VALIDATION_SHARE = 0.3
OUTPUT_PATH = '/kaggle/working/'
TRAIN_CSV_INPUT_PATH = "'/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv"
TEST_CSV_INPUT_PATH = '/kaggle/input/llm-detect-ai-generated-text/test_essays.csv'

# Training

In [ ]:
model_checkpoint = "/kaggle/input/huggingfacedebertav3variants/deberta-v3-large"
max_length = 512
model_name = "deberta-v3-large"

In [ ]:
def get_df() -> pd.DataFrame:
    daigt = pd.read_csv(TRAIN_CSV_INPUT_PATH)
    daigt.rename(columns={'label': 'generated'}, inplace=True)
    df_ = daigt[daigt['RDizzl3_seven'] == True] # only True ones
    cols = ['text', 'generated']
    df_ = df_[cols]
    return df_

def ds_prep(df: pd.DataFrame, y: pd.Series) -> Dataset:
    df_ = df.copy()
    df_ = (df_.loc[y.index.to_list()]
              .rename(columns={'generated': 'label'})
          )
    
    return Dataset.from_pandas(df_)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
    auc = roc_auc_score(labels, probs[:,1], multi_class='ovr')
    return {"roc_auc": auc}

def get_probas_with_trainer(ds_enc: Dataset) -> Dataset:
    
    ds_preds = trainer.predict(ds_enc)
    logits = ds_preds.predictions
    probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
    
    # keeping only positives proba  
    return probs[:,1] 

def to_df_from_ds(ds: Dataset, y: pd.DataFrame) -> pd.DataFrame:
    
    df_ = ds.to_pandas()
    df_.index(y.index, inplace=True)
    
    return df_

In [ ]:
df = get_df()

df = df.rename(columns={'generated': 'label'})
y = df['label']

df_tr, df_val = train_test_split(df, test_size=VALIDATION_SHARE ,stratify=y, random_state=SEED)

# creating Dataset format 
ds_tr = Dataset.from_pandas(df_tr)
ds_val = Dataset.from_pandas(df_val)

train_obs = df_tr.shape[0]
val_obs = df_val.shape[0]

del df_tr, df_val
gc.collect()

# loading tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def preprocess_function(ds: Dataset, text_col: str = 'text'):
    return tokenizer(ds[text_col], max_length=max_length, padding=True, truncation=True)

# encoding
ds_tr_enc = ds_tr.map(preprocess_function, batched=True)

ds_val_enc = ds_val.map(preprocess_function, batched=True)

# choosing pretrained model
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

# training loop params
metric_name = "roc_auc"
train_batch_size = 2 # 8
eval_batch_size = 8 # 32
grad_acc = 4
num_steps = train_obs // (train_batch_size * grad_acc)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = num_steps // 3,
    save_steps = num_steps // 3,
    learning_rate=2e-5,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    gradient_accumulation_steps=grad_acc,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=False,
    metric_for_best_model=metric_name,
    report_to='none' # change to wandb when internet access is enabled
)

trainer = Trainer(
    model.to(device),
    args,
    train_dataset=ds_tr_enc,
    eval_dataset=ds_val_enc,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.save_model(OUTPUT_PATH)

def get_probas_with_trainer(ds_enc: Dataset) -> Dataset:
    
    ds_preds = trainer.predict(ds_enc)
    logits = ds_preds.predictions
    probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
    
    # keeping only positives proba  
    return probs[:,1] 

ds_tr_proba = get_probas_with_trainer(ds_tr_enc)
ds_val_proba = get_probas_with_trainer(ds_val_enc)

# Testing

In [ ]:
df_test = pd.read_csv(TEST_CSV_INPUT_PATH)

df_test = df_test[['id', 'text']]
df_test_id = df_test['id']

ds_test = Dataset.from_pandas(df_test)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

ds_test_enc = ds_test.map(preprocess_function, batched=True)
ds_test_proba = get_probas_with_trainer(ds_test_enc)

## Submission

In [ ]:
results = pd.DataFrame({'id': df_test_id, 'generated': ds_test_proba})
results.to_csv('submission.csv', index=False)